In [10]:
library(RTextTools)

Loading required package: SparseM


Attaching package: 'SparseM'


The following object is masked from 'package:base':

    backsolve




In [1]:
# Read the data
data <- read.csv('data/sunnyData.csv')

In [2]:
data

Text,IsSunny
<chr>,<int>
sunny,1
rainy,-1
sunny sunny,1
sunny rainy,-1
rainy sunny,-1
rainy rainy,-1
sunny sunny sunny,1
sunny rainy sunny,1
sunny sunny rainy,1


In [11]:
# Create the document term matrix
dtMatrix <- create_matrix(data["Text"])

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


In [12]:
dtMatrix

<<DocumentTermMatrix (documents: 11, terms: 2)>>
Non-/sparse entries: 17/5
Sparsity           : 23%
Maximal term length: 5
Weighting          : term frequency (tf)

Create and train the SVM model

In [16]:
# Configure the training data
container <- create_container(dtMatrix, data$IsSunny, trainSize=1:11, virgin=FALSE)

In [17]:
container

An object of class "matrix_container"
Slot "training_matrix":
An object of class "matrix.csr"
Slot "ra":
 [1] 1 1 2 1 1 1 1 2 3 1 2 1 2 1 2 2 1

Slot "ja":
 [1] 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2

Slot "ia":
 [1]  1  2  3  4  6  8  9 10 12 14 16 18

Slot "dimension":
[1] 11  2


Slot "classification_matrix":
An object of class "matrix.csr"
Slot "ra":
 [1] 1 1 2 1 1 1 1 2 3 1 2 1 2 1 2 2 1

Slot "ja":
 [1] 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2

Slot "ia":
 [1]  1  2  3  4  6  8  9 10 12 14 16 18

Slot "dimension":
[1] 11  2


Slot "training_codes":
 [1] 1  -1 1  -1 -1 -1 1  1  1  1  -1
Levels: -1 1

Slot "testing_codes":
 [1] 1  -1 1  -1 -1 -1 1  1  1  1  -1
Levels: -1 1

Slot "column_names":
[1] "rainy" "sunny"

Slot "virgin":
[1] FALSE


In [ ]:
# Train a SVM Model
model <- train_model(container, "SVM", kernel="linear", cost=1)

In [18]:
model


Call:
svm.default(x = container@training_matrix, y = container@training_codes, 
    kernel = kernel, cost = cost, cross = cross, probability = TRUE, 
    method = method)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  4


### Predict with new data

In [20]:
# New data
predictionData <- list("sunny sunny sunny rainy rainy", "rainy sunny rainy rainy", "hello", "", "this is another rainy world")

In [21]:
predictionData

[[1]]
[1] "sunny sunny sunny rainy rainy"

[[2]]
[1] "rainy sunny rainy rainy"

[[3]]
[1] "hello"

[[4]]
[1] ""

[[5]]
[1] "this is another rainy world"

Create a document term matrix for the test data:

In [22]:
# Create a prediction document term matrix
predMatrix <- create_matrix(predictionData, originalMatrix=dtMatrix)


Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


In [23]:
predMatrix

<<DocumentTermMatrix (documents: 5, terms: 2)>>
Non-/sparse entries: 5/5
Sparsity           : 50%
Maximal term length: 5
Weighting          : term frequency (tf)

In [25]:
# Create the corresponding container
predSize = length(predictionData);
predictionContainer <- create_container(predMatrix, labels=rep(0,predSize), testSize=1:predSize, virgin=FALSE)

In [26]:
# Predict
results <- classify_model(predictionContainer, model)
results

,SVM_LABEL,SVM_PROB
,<chr>,<dbl>
1,1,0.7410242
2,-1,0.8654268
3,1,0.5201127
4,1,0.5201127
5,-1,0.7089542
